# Python Simple Trading Algorithm assignment
*this was working last time i checked it*

> import files

In [7]:
from alpha_vantage.timeseries import TimeSeries
# from pyalgotrading import plot_candles
import pandas
import copy

> Script class

In [8]:

class ScriptData:
    def __init__(self):
        self.key = TimeSeries(key="Y455ZZ06T6P3LRSW", output_format = 'pandas', indexing_type='integer')
        self.df = {}
        pass

    def __getitem__(self,script):
        if script in self.df:
            return self.df[script]
        else:
            return "not found"

    def fetch_intraday_data(self,script):
        self.df[script], mata_data = self.key.get_intraday(script)
        pass
    def convert_intray_data(self,script):
        self.df[script].rename(columns={"index":"timestamp", "1. open": "open","2. high":"high","3. low":"low","4. close":"close","5. volume":"volume"}, inplace=True)
        pass

> Indicator function

In [9]:
def indicator1(df,timeperiod:int):
    df.drop(columns=["open","high","low","volume"],inplace=True)
    df.rename(columns={"close":"indicator"},inplace=True)
    df_new = copy.copy(df)

    for i in range(0,len(df["timestamp"])):
        if i<= timeperiod:
            df_new.at[i,"indicator"] = "NAN"
        else:
            try:
                df_new.at[i,"indicator"] =round( sum(df["indicator"].iloc[i-timeperiod : i])/timeperiod , 2)
            except:
                print(i,type(i))
    return df_new

> `optional`  test code to run functions completed rn

In [10]:
# obj = ScriptData()
# obj.fetch_intraday_data("GOOGL")
# obj.convert_intray_data("GOOGL")
# print(obj["GOOGL"][:15])
# print(indicator1(obj['GOOGL'],5)[:15])

> Strategy class `with fallback and debuging code`

In [11]:
class Strategy:
    def __init__(self,script):
        self.id = script
        self.data = None
        self.predictor = None
        self.signal = None
        self.timeperiod = 5

    def get_script_data(self):
        data_obj = ScriptData()
        data_obj.fetch_intraday_data(self.id)
        data_obj.convert_intray_data(self.id)
        self.data = data_obj[self.id]
        self.predictor = indicator1(copy.copy(self.data),self.timeperiod)

        self.signal = copy.copy(self.predictor)
        self.signal.rename(columns={"indicator":"signal"},inplace=True)

        for i in range(self.timeperiod+1,len( self.signal["timestamp"])-1):
            
            try:
                # print(self.data["close"].iloc[i+1])
                # print(self.data["close"].iloc[i-1])
                # print(self.data["close"].iloc[i])
                # print(self.predictor["indicator"].iloc[i])

                if (self.data["close"].iloc[i+1] < self.predictor["indicator"].iloc[i] or self.data["close"].iloc[i-1] > self.predictor["indicator"].iloc[i]) and self.data["close"].iloc[i] == self.predictor["indicator"].iloc[i]:
                    self.signal.at[i,"signal"] = "BUY"
                    # print("buy")

                elif (self.data["close"].iloc[i+1] > self.predictor["indicator"].iloc[i] or self.data["close"].iloc[i-1] < self.predictor["indicator"].iloc[i]) and self.data["close"].iloc[i] == self.predictor["indicator"].iloc[i]:
                    self.signal.at[i,"signal"] = "SELL"
                    # print("SELL")

                elif self.data["close"].iloc[i] != self.predictor["indicator"].iloc[i]:
                    self.signal.at[i,"signal"] = "NO_SIGNAL"

                else:
                    print("did not work")      
            except:
                print(i)
    def get_signals(self):
        return self.signal


> Finaly code to test all this

In [12]:
strategy = Strategy("NVDA")
strategy.get_script_data()
print(strategy.get_signals())
# df = strategy.get_signals()
# plot_candles(df)

                 timestamp     signal
index                                
0      2023-02-14 20:00:00        NAN
1      2023-02-14 19:45:00        NAN
2      2023-02-14 19:30:00        NAN
3      2023-02-14 19:15:00        NAN
4      2023-02-14 19:00:00        NAN
...                    ...        ...
95     2023-02-13 12:15:00  NO_SIGNAL
96     2023-02-13 12:00:00  NO_SIGNAL
97     2023-02-13 11:45:00  NO_SIGNAL
98     2023-02-13 11:30:00  NO_SIGNAL
99     2023-02-13 11:15:00     219.03

[100 rows x 2 columns]
